# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support

In [2]:
# downloading nltk libraries

# # The line is commented to avoid reinstallation
# nltk.download("popular")

# # Done downloading collection popular

In [9]:
# load data from database
engine = create_engine('sqlite:///NesrCategories.db')
sql = 'SELECT * FROM CategorizedMessages'
df = pd.read_sql(sql, engine)
X = df.message
Y = df.iloc[:, 4:]

# Get Y labels
Y_labels = list(Y)

In [4]:
# Check
display(df.shape, X.shape, Y.shape, 
        df.head(1), X.head(2), Y.head(2))

(26216, 40)

(26216,)

(26216, 36)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


0    Weather update - a cold front from Cuba that c...
1              Is the Hurricane over or is it not over
Name: message, dtype: object

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [27]:
# Define some shortcuts
stop_words = stopwords.words("english")
lemmatizer = WordNetLemmatizer()

def tokenize(text):
    # Normalize text
    text = text.lower()
    text = re.sub(r"[^a-z0-9]", " ", text)
    # Tokenize text
    tokens = word_tokenize(text)
    # lemmatize and Remove stop words
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    # Return
    return tokens

def display_results(y_test, y_pred):
    labels = np.unique(y_pred)
    print("Labels: ", labels)
    confusion_mat = confusion_matrix(y_test, y_pred, labels=labels)
    print("\nConfusion Matrix:\n", confusion_mat)
    accuracy = (y_pred == y_test).mean()
    print(f"\nAccuracy: {accuracy *100:5.2f}%\n")
    weig = precision_recall_fscore_support(y_test, y_pred, average='weighted')
    micr = precision_recall_fscore_support(y_test, y_pred, average='micro')
    macr = precision_recall_fscore_support(y_test, y_pred, average='macro')
    #bina = precision_recall_fscore_support(y_test, y_pred, average='binary')
    print(f"{'Metric':<12}{'Weighted':>10}{'Micro':>10}{'Macro':>10}") #{'Binary':>10}")
    print(f"{'Precision:':<12}{weig[0] *100:9.2f}%{micr[0] *100:9.2f}%{macr[0] *100:9.2f}%")
    print(f"{'Recall:':<12}{weig[1] *100:9.2f}%{micr[1] *100:9.2f}%{macr[1] *100:9.2f}%")
    print(f"{'F Score:':<12}{weig[2] *100:9.2f}%{micr[2] *100:9.2f}%{macr[2] *100:9.2f}%")

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

#### 3-1 Trying to build normal Sequence of ML
> *Trying on **one output** only*

In [28]:
# Extracting one output only
y = Y.iloc[:, 0]

# Split data to train and test
print ('Splitting...')
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Defining vectorizers and classifiers
print ('Defining tokenizers...')
vect = CountVectorizer(tokenizer=tokenize)
tfidf = TfidfTransformer()
clf = RandomForestClassifier(n_estimators=100)

# train classifier
print ('Vectorizing...')
X_train_counts = vect.fit_transform(X_train)
print ('Performing Tfidf...')
X_train_tfidf = tfidf.fit_transform(X_train_counts)
print ('Performing Classification...')
clf.fit(X_train_tfidf, y_train)

# predict on test data
print ('Performing Predictions...')
X_test_counts = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_counts)
y_pred = clf.predict(X_test_tfidf)

# display results
print ('Scoring...')
display_results(y_test, y_pred)

Splitting...
Defining tokenizers...
Vectorizing...
Performing Tfidf...
Performing Classification...
Performing Predictions...
Scoring...
Labels:  [0 1 2]

Confusion Matrix:
 [[ 672  848   24]
 [ 253 4683   36]
 [   1   18   19]]

Accuracy: 82.00%

Metric        Weighted     Micro     Macro
Precision:      81.26%    82.00%    60.34%
Recall:         82.00%    82.00%    62.57%
F Score:        80.54%    82.00%    58.64%


In [ ]:
pipeline = 

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.